## Семинар 3 "Определение ключевых точек лица"

## Задание

Используя датасет из архива решите задачу регрессии для поиска 68-ми ключевых точек лица. 

Для обучения и валидации используйте изображения и разметку из папок train и test соответственно.

Для **зачета** дз нужно сделать одно из двух:

1. Исследовать 6 разных архитектур и написать подробный вывод + мысли почему одна сеть работает лучше чем другая (разные функции активации, глубины сетей, размеры и типы слоев)
2. Преодолеть порог MSE=8 на тесте


Также до  22 марта 23:59 можно получить до 5 дополнительных баллов за лучшее решение. (За первое место 5 баллов, за второе — 3 балла, за третье — 1 балла.)

Для решения можно пользоваться всем инструментарием, с которым мы познакомимся:  предобученные модели, сверточные сети, любые оптимизаторы, batchnorm, dropout, etc.

In [305]:
%matplotlib inline
import os
import torch
import sys
import pandas as pd
from skimage import io, transform
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.nn import MSELoss, Sequential, Linear, Sigmoid, Tanh, L1Loss, Module, Parameter
from torch.nn import Flatten,Dropout, LayerNorm, Sequential,\
                        Module, Conv2d, BatchNorm2d, ReLU, MaxPool2d, BatchNorm1d
from torch.autograd import Variable
# Ignore warnings
import warnings

base_dir = './'

try:
    from tensorboardX import SummaryWriter
except ImportError:
    raise RuntimeError("No tensorboardX package is found. Please install with the command: \npip install tensorboardX")

from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss

In [367]:
warnings.filterwarnings("ignore")

landmarks_frame = pd.read_csv(base_dir + 'dataset/train/face_landmarks.csv')

n = 100
img_name = landmarks_frame.iloc[n, 0]
landmarks = landmarks_frame.iloc[n, 1:].values.astype('float')
landmarks = landmarks.reshape(-1, 2)

print('Image name: {}'.format(img_name))
print('Landmarks shape: {}'.format(landmarks.shape))
print('First 4 Landmarks: {}'.format(landmarks[:4]))
print('Image shape {}'.format( io.imread(os.path.join(base_dir + 'dataset/train/', img_name)).shape  ) )

def show_landmarks(image, landmarks):
    """Show image with landmarks"""
    plt.imshow(image, cmap='gray')
    plt.scatter(landmarks[:, 0], landmarks[:, 1], s=10, marker='.', c='red', cmap='rgb')
    plt.pause(0.001)  

plt.figure()
img = io.imread(os.path.join(base_dir + 'dataset/train/', img_name))
show_landmarks(img, landmarks)
plt.show()

In [307]:
def show_landmarks_batch(sample_batch, y_pred=None):
    """Show image with landmarks for a batch of samples."""
    """ y_pred: (batch_size, fp_num, 2) """
    images_batch, landmarks_batch = sample_batch
    batch_size = len(images_batch)
    im_size = images_batch.size(2)

    grid = utils.make_grid(images_batch)
    print(images_batch.numpy().shape)
    plt.imshow(grid.numpy().transpose(1,0,2).reshape(im_size, -1), cmap="gray")

    for i in range(batch_size):
        plt.scatter(landmarks_batch[i, :, 0].numpy() + i * im_size,
                    landmarks_batch[i, :, 1].numpy(),
                    s=10, marker='.', c='r', label='Real')

    if type(y_pred) != type(None):
        for i in range(batch_size):
            plt.scatter(y_pred[i, :, 0] + i * im_size,
                        y_pred[i, :, 1],
                        s=10, marker='.', c='b',  label='Prediction')
        plt.title('Batch from dataloader')
    plt.legend()

In [252]:
img1, lm1 = Rescale(144)((img, landmarks))
show_landmarks(img1, lm1)

In [254]:
img3, lm3 = RandomCrop(128)((img1, lm1))
show_landmarks(img3, lm3)

In [259]:
img2, lm2 = RandomRotate(70)((img3, lm3))
show_landmarks(img2, lm2)

In [360]:
plt.imshow(img, cmap='gray')

In [372]:
from skimage import filters
img = io.imread(os.path.join(base_dir + 'dataset/train/', img_name))
img = filters.gaussian(img, sigma=0)
plt.imshow(img, cmap='gray')

In [378]:
from skimage.util import random_noise
img = io.imread(os.path.join(base_dir + 'dataset/train/', img_name))
img = random_noise(img, mode='s&p', amount=0.01, clip=True)
plt.imshow(img, cmap='gray')

In [308]:
from skimage import filters
from skimage.util import random_noise

class RandomBlur(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, sigma_range=2.0):
        assert isinstance(sigma_range, (int))
        self.sigma_range = sigma_range
        

    def __call__(self, sample):
        image, landmarks = sample
        
        sigma = np.random.uniform(0.0, self.sigma_range, 1)
        image = filters.gaussian(image, sigma=sigma)

        return image, landmarks


class RandomSP(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, amount_range=0.001):
        assert isinstance(sigma_range, (int))
        self.amount_range = amount_range
        
        
    def __call__(self, sample):
        image, landmarks = sample
        
        amount = np.random.uniform(0.0, self.amount_range, 1)
        image = random_noise(image, mode='s&p', amount=amount, clip=True)

        return image, landmarks



class Resize(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size
        

    def __call__(self, sample):
        image, landmarks = sample

        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        shift = (new_h - h) // 2
        img = np.pad(image, shift, mode='constant')
        landmarks = landmarks + [shift, shift]

        return img, landmarks


    
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, landmarks = sample

        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (new_h, new_w))
    
        # h and w are swapped for landmarks because for images,
        # x and y axes are axis 1 and 0 respectively
        landmarks = landmarks * [new_w / w, new_h / h]

        return img, landmarks


class RandomCrop(object):
    """Crop randomly the image in a sample.

    Args:
        output_size (tuple or int): Desired output size. If int, square crop
            is made.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    
    def __call__(self, sample):
        image, landmarks = sample

        h, w = image.shape[:2]
        new_h, new_w = self.output_size

        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)

        image = image[top: top + new_h,
                      left: left + new_w]

        landmarks = landmarks - [left, top]

        return image, landmarks
    

class RandomRotate(object):
    # img, facepoints, alpha_degree
    
    def __init__(self, degree_range):
        assert isinstance(degree_range, (int))
        assert degree_range >= 1
        self.degree_range = degree_range
    
    
    def __call__(self, sample):
        image, landmarks = sample
        
        alpha_degree = np.random.randint(-self.degree_range, self.degree_range, 1)[0]
        new_img = transform.rotate(image, alpha_degree, mode='constant') # mode='edge'
        
        # center = (image.shape[0] / 2 - 0.5, image.shape[1] / 2 - 0.5) # как в skimage.transform.rotate
        # center = (image.shape[0] // 2, image.shape[1] // 2)
        center = (image.shape[0] / 2, image.shape[1] / 2)
        landmarks = np.copy(landmarks)
        
        vector = landmarks - center
        alpha_radian = -np.radians(alpha_degree)
        rm = np.array([[np.cos(alpha_radian), -np.sin(alpha_radian)],
                      [np.sin(alpha_radian), np.cos(alpha_radian)]], dtype='float32')
        
        vector = (rm.dot(vector.T)).T
        
        landmarks = (vector + center).astype(dtype=int)
        
        return new_img, landmarks


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, landmarks = sample
        return torch.from_numpy(image).type(torch.float), torch.from_numpy(landmarks).type(torch.float)

In [309]:
class FaceLandmarksDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, device, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.device = device

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.landmarks_frame.iloc[idx, 0])
        image = io.imread(img_name)
        landmarks = self.landmarks_frame.iloc[idx, 1:].values.astype('float')
        landmarks = landmarks.reshape(-1, 2)

        assert len(image.shape) == 2
            
        if self.transform:
            image, landmarks = self.transform((image, landmarks))
        
        image = image.reshape(1, *image.shape)
        landmarks = landmarks.reshape(68 * 2)
        sample = image.to(self.device), landmarks.to(self.device)

        return sample

In [319]:
train_dataset = FaceLandmarksDataset(csv_file=base_dir + 'dataset/train/face_landmarks.csv',
                                     root_dir=base_dir + 'dataset/train',
                                     device='cpu',
                                     transform=transforms.Compose([ # transforms.Compose  Sequential
                                                Rescale(128), # 192 128
                                                # RandomCrop(128),
                                                RandomRotate(25),
                                               #RandomCrop(127),
                                                ToTensor()
                                           ]))

In [320]:
img, tg = train_dataset[0]
img = img[0]
img.shape

In [321]:
# io.imshow(img.numpy())
show_landmarks(img.numpy(), tg.numpy().reshape((68, 2)))

In [322]:
device = "cpu"

if torch.cuda.is_available():
    device = "cuda"

print(device)

In [379]:
train_dataset = FaceLandmarksDataset(csv_file=base_dir + 'dataset/train/face_landmarks.csv',
                                     root_dir=base_dir + 'dataset/train',
                                     device=device,
                                     transform=transforms.Compose([ # transforms.Compose  Sequential
                                                Rescale(128), # 192 128 144
                                                # RandomCrop(128),
                                                # RandomRotate(25), #70
                                               #RandomCrop(127),
                                                ToTensor()
                                           ]))
#                                      transform=transforms.Compose([
#                                                Rescale(100),
#                                                RandomCrop(96),
#                                                ToTensor()
#                                            ]))

test_dataset = FaceLandmarksDataset(csv_file=base_dir + 'dataset/test/face_landmarks.csv',
                                     root_dir=base_dir + 'dataset/test',
                                     device=device,
                                     transform=transforms.Compose([
                                               Rescale(128),
                                               #RandomCrop(96),
                                               ToTensor()
                                           ]))

In [386]:
res = []
for i in tqdm(range(len(train_dataset))):
    res.append(train_dataset[i][0][0].numpy())

In [387]:
res.shape

In [388]:
np.mean(res)

In [389]:
np.std(res)

In [324]:
def get_data_loaders(train_batch_size=16, val_batch_size=16): # 64

    train_loader = DataLoader(
        train_dataset, batch_size=train_batch_size, shuffle=True
    )

    test_loader = DataLoader(
        test_dataset, batch_size=val_batch_size, shuffle=False
    )

    return train_loader, test_loader

In [333]:
class Net(Module):

    def __init__(self, input_shape=(1, 128, 128)):
        """ input_shape = (c, h, w)"""
        super().__init__()

        # нормализация?
        
        self.cv1 = Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1) 
        self.ac1 = ReLU()
        self.bn1 = BatchNorm2d(32)
        self.ap1 = MaxPool2d(kernel_size=2) 
        
        self.cv2 = Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1) 
        self.ac2 = ReLU()
        self.bn2 = BatchNorm2d(64)
        
        self.cv3 = Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1) 
        self.ac3 = ReLU()
        self.bn3 = BatchNorm2d(64)
        self.ap2 = MaxPool2d(2)
        
        self.cv4 = Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1) 
        self.ac4 = ReLU()
        self.bn4 = BatchNorm2d(64)
        
        self.cv5 = Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1) 
        self.ac5 = ReLU()
        self.bn5 = BatchNorm2d(64)
        self.ap3 = MaxPool2d(2)
        
        self.cv6 = Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1) 
        self.ac6 = ReLU()
        self.bn6 = BatchNorm2d(128)
        
        self.cv7 = Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1) 
        self.ac7 = ReLU()
        self.bn7 = BatchNorm2d(128)
        self.ap4 = MaxPool2d(2)
        
        self.cv8 = Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1) 
        self.ac8 = ReLU()
        self.bn8 = BatchNorm2d(256)

        self.ln1 = Linear(256 * 8 * 8, 1024) # 32 * 6 * 6
        self.ac9 = ReLU()
        self.bn9 = BatchNorm1d(1024)

        self.ln2 = Linear(1024, 68 * 2)

        self.layers = [
            
            self.cv1,
            self.ac1,
            self.bn1,
            self.ap1,

            self.cv2,
            self.ac2,
            self.bn2,

            self.cv3,
            self.ac3,
            self.bn3,
            self.ap2,

            self.cv4, 
            self.ac4,
            self.bn4,

            self.cv5, 
            self.ac5,
            self.bn5,
            self.ap3,

            self.cv6,
            self.ac6,
            self.bn6,

            self.cv7,
            self.ac7,
            self.bn7,
            self.ap4,

            self.cv8,
            self.ac8,
            self.bn8,
            
            lambda y: y.view(-1, y.shape[-1] * y.shape[-2] * y.shape[-3]),

            self.ln1,
            self.ac9,
            self.bn9,

            self.ln2
        ]
        
    def forward(self, x):
        
        for layer in self.layers:
            x = layer(x)
        
        return x

In [334]:
network = Net()

train_dataloader, test_dataloader = get_data_loaders()
writer = SummaryWriter(filename_suffix='first', logdir="logs")
    
criterion = MSELoss()
optim = torch.optim.Adam
learning_rate = 0.01
max_epochs = 15 # 20

optimizer = optim(network.parameters())#, lr=learning_rate)

trainer = create_supervised_trainer(network, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(
    network, metrics={"L2 loss": Loss(criterion), "L1 loss": Loss(L1Loss())}, device=device
)

training_history = {'l1':[],'l2':[]}
validation_history = {'l1':[],'l2':[]}
last_epoch = []

In [335]:
@trainer.on(Events.ITERATION_COMPLETED(every=10))
def log_training_loss(engine):
    writer.add_scalar("training/loss", engine.state.output, engine.state.iteration)
    writer.flush()


@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    evaluator.run(train_dataloader)
    metrics = evaluator.state.metrics
    l2_train = metrics["L2 loss"]
    l1_train = metrics['L1 loss']
    last_epoch.append(0)
    training_history['l2'].append(l2_train)
    training_history['l1'].append(l1_train)
    writer.add_scalar("training/epoch_loss", l2_train, trainer.state.epoch)
    
    evaluator.run(test_dataloader)
    metrics = evaluator.state.metrics
    l2 = metrics["L2 loss"]
    l1 = metrics['L1 loss']
    validation_history['l2'].append(l2)
    validation_history['l1'].append(l1)
    
    writer.add_scalar("validation/epoch_loss", l2, trainer.state.epoch)
    writer.flush()
    
    sys.stdout.write("\rTraining/Validation Results - Epoch: {}  Avg MSE: {:.2f} / {:.2f} Avg MAE: {:.2f} / {:.2f}"
          .format(trainer.state.epoch, l2_train, l2, l1_train, l1))
   

In [336]:
trainer.run(train_dataloader, max_epochs=max_epochs)
# TODO: мб убрать вращение, из-за этого скор ужасно падает

In [280]:
plt.figure(figsize=(12, 5))
plt.plot(training_history['l2'][1:], label='Train')
plt.plot(validation_history['l2'][1:], label='Test')
plt.xlabel('Epochs', fontsize=16)
plt.ylabel('Loss', fontsize=16)
plt.legend(loc=0, fontsize=16)
plt.grid('on')
plt.show()

In [284]:
_, test_dataloader = get_data_loaders(val_batch_size=2)

# images_batch = next(iter(test_dataloader))
# y_pred = network(images_batch[0]).detach().numpy()
# y_pred.shape

In [285]:
images_batch = next(iter(test_dataloader))
x, y = images_batch

# y_pred = network(images_batch[0]).detach().numpy()
y_pred = network(x)

y_pred = y_pred.data.numpy()
show_landmarks_batch(images_batch, y_pred)
show_landmarks_batch(images_batch, y_pred)

In [ ]:
class Net(Module):

    def __init__(self, input_shape=(1, 96, 96)):
        """ input_shape = (c, h, w)"""
        super().__init__()

        #self.bn1 = BatchNorm2d(1)
        # self.ly1 = LayerNorm((1, 128, 128)) # (1, 96, 96)
        self.cv1 = Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1) 
        self.ac1 = ReLU()
        self.ap1 = MaxPool2d(2)

        self.ly2 = LayerNorm((16, 64, 64)) # (16, 48, 48)
        self.cv2 = Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.ac2 = ReLU()
        self.ap2 = MaxPool2d(2)

        self.ly3 = LayerNorm((32, 32, 32)) # (32, 24, 24)
        self.cv3 = Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.ac3 = ReLU()
        self.ap3 = MaxPool2d(2)

        self.ly4 = LayerNorm((64, 16, 16)) # (64, 12, 12)
        self.cv4 = Conv2d(in_channels=64, out_channels=32, kernel_size=3, padding=1)
        self.ac4 = ReLU()
        self.ap4 = MaxPool2d(2) # 6 * 6

        self.ly5 = LayerNorm((32, 8, 8)) # (32, 6, 6)
        self.dp1 = Dropout(0.5)

        self.ln1 = Linear(32 * 8 * 8, 512) # 32 * 6 * 6
        self.ac5 = ReLU()
        self.bn1 = BatchNorm1d(512)
        self.dp2 = Dropout(0.5)

        self.ln2 = Linear(512, 256)
        self.ac6 = ReLU()
        self.bn2 = BatchNorm1d(256)
        self.dp3 = Dropout(0.5)

        self.ln3 = Linear(256, 68 * 2)

        self.layers = [
            self.ly1,
            self.cv1,
            self.ac1,
            self.ap1,

            self.ly2,
            self.cv2,
            self.ac2,
            self.ap2,

            self.ly3,
            self.cv3,
            self.ac3,
            self.ap3,

            self.ly4,
            self.cv4,
            self.ac4,
            self.ap4,
            
            self.ly5,
            lambda y: y.view(-1, y.shape[-1] * y.shape[-2] * y.shape[-3]),
            self.dp1,

            self.ln1,
            self.ac5,
            self.bn1,
            self.dp2,

            self.ln2,
            self.ac6,
            self.bn2,
            self.dp3,

            self.ln3
        ]
        
    def forward(self, x):
        
        for layer in self.layers:
            x = layer(x)
        
        return x